# Alur Preprocessing 1 (Text feature Extraction - POS-Tagging)

Alur dalam preprocessing ini yaitu dengan mengekstraksi kata-kata dengan beberapa metode. Alurnya adalah sebagai berikut

1. Load Data
2. POS-Tagging
4. Vectorize
5. Analyze

In [1]:
import matplotlib.pyplot as plt
import os, re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from datetime import datetime as dt
import cPickle as pickle
import re #Regex
%matplotlib inline

In [4]:
#just load it from the pickle dump
DATA_JUDUL = pickle.load( open( "v1.4\DATA_JUDULStriped-v4a.p", "rb" ) )
DATA_SHORT = pickle.load( open( "v1.4\DATA_SHORTStriped-v4a.p", "rb" ) )
DATA_LONG = pickle.load( open( "v1.4\DATA_LONGStriped-v4a.p", "rb" ) )
DATA_TARGET = pickle.load( open( "v1.4\DATA_TARGET-v4a.p", "rb" ) )

#just load it from the pickle dump
DATA_JUDUL_sk = pickle.load( open( "v1.4\DATA_JUDULStriped-v4c.p", "rb" ) )
DATA_SHORT_sk = pickle.load( open( "v1.4\DATA_SHORTStriped-v4c.p", "rb" ) )
DATA_LONG_sk = pickle.load( open( "v1.4\DATA_LONGStriped-v4c.p", "rb" ) )

In [5]:
len(DATA_TARGET)

1344

In [3]:
%time postagger = pickle.load(open( "POSTAGGER.p", "rb" ))

Wall time: 13.9 s


In [24]:
import string

def onlyAZ(teks):
    return re.sub(r'[^a-zA-Z]',' ', teks)
    
def onlyNVFromSentence(teks):
    splitted = postagger.tag(onlyAZ(teks).split())
    nouns = [word for word,pos in splitted \
        if (pos == 'NN' or pos == 'NNP' or 
            pos == 'NNS' or pos == 'VB')]
    nounsSentence = string.join(nouns)
    return nounsSentence

def onlyNVFromParagraph(par):
    splittedPar = par.split('.')
    nounsPar = string.join([onlyNVFromSentence(i) for i in splittedPar])
    return  nounsPar.strip() 

In [5]:
#Time saver strategy
%time DATA_JUDULPos = DATA_JUDUL.map(lambda x: onlyNVFromParagraph(x))
%time DATA_SHORTPos = DATA_SHORT.map(lambda x: onlyNVFromParagraph(x))
%time DATA_LONGPos  = DATA_LONG.map(lambda x: onlyNVFromParagraph(x))

%time DATA_JUDUL_skPos = DATA_JUDUL_sk.map(lambda x: onlyNVFromParagraph(x))
%time DATA_SHORT_skPos = DATA_SHORT_sk.map(lambda x: onlyNVFromParagraph(x))
%time DATA_LONG_skPos  = DATA_LONG_sk.map(lambda x: onlyNVFromParagraph(x))

Wall time: 77 ms
Wall time: 209 ms
Wall time: 2.4 s
Wall time: 7 ms
Wall time: 15 ms
Wall time: 173 ms


In [6]:
print DATA_SHORT[DATA_SHORT.index[0]]
print DATA_SHORTPos[DATA_SHORTPos.index[0]]

saat masih lajang memang kebutuhan tak terlalu mendesak.
lajang kebutuhan mendesak


## Vektorisasi

##### Contoh Vektorisasi

In [7]:
#Stopwords data source : https://www.illc.uva.nl/Research/Publications/Reports/MoL-2003-02.text.pdf
stopword_html = open("id.stopwords.01.01.2016.txt",'r').read()
stopwords     = stopword_html.split("\n")
print stopwords

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [8]:
import pandas as pd
import numpy as np
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])
#countv = TfidfVectorizer(stop_words=stopwords,decode_error='ignore')
countv = TfidfVectorizer()
dummy  = countv.fit_transform(docs)

In [9]:
import pandas as pd
import numpy as np
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])
n_docs = len(docs)
countv = CountVectorizer()
dummy  = countv.fit_transform(docs)
# examine the vocabulary and document-term matrix together
pd.DataFrame(dummy.toarray(), columns=countv.get_feature_names())

,and,is,shining,sun,sweet,the,weather
0,0,1,1,1,0,1,0
1,0,1,0,0,1,1,1
2,1,2,1,1,1,2,1


In [10]:
tf_shining = 1.00
df_shining = 2.00
tf_shining*(np.log((n_docs+1)/(df_shining+1)) + 1)

1.2876820724517808

In [11]:
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])
countv = TfidfVectorizer(norm=None)
dummy  = countv.fit_transform(docs)

In [12]:
# examine the vocabulary and document-term matrix together
dfDummy = pd.DataFrame(dummy.toarray(), columns=countv.get_feature_names())
dfDummy

,and,is,shining,sun,sweet,the,weather
0,0.000000,1,1.287682,1.287682,0.000000,1,0.000000
1,0.000000,1,0.000000,0.000000,1.287682,1,1.287682
2,1.693147,2,1.287682,1.287682,1.287682,2,1.287682


In [13]:
smooth_tfidfs_d3 = np.array([[ 1.69314718, 2.0, 1.28768207, 1.28768207, 1.28768207, 2.0, 1.28768207]])
smooth_tfidfs_d3_norm = smooth_tfidfs_d3/np.sqrt(np.sum(smooth_tfidfs_d3**2))
smooth_tfidfs_d3_norm

array([[ 0.40474829,  0.47810172,  0.30782151,  0.30782151,  0.30782151,
         0.47810172,  0.30782151]])

In [14]:
'The sun is shining',
'The weather is sweet',
'The sun is shining and the weather is sweet'

,and,is,shining,sun,sweet,the,weather
0,0.000000,0.433708,0.558478,0.558478,0.000000,0.433708,0.000000
1,0.000000,0.433708,0.000000,0.000000,0.558478,0.433708,0.558478
2,0.404748,0.478102,0.307822,0.307822,0.307822,0.478102,0.307822


In [ ]:
import pandas as pd
import numpy as np
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining and the weather is sweet'])
n_docs = len(docs)
countv = TfidfVectorizer()
dummy  = countv.fit_transform(docs)
# examine the vocabulary and document-term matrix together
pd.DataFrame(dummy.toarray(), columns=countv.get_feature_names())

In [15]:
def rmStop(teks):
    for i in stopwords:
        teks = teks.replace(' '+i+' ',' ')
    return len(teks.split())

def rmStops(teks):
    for i in stopwords:
        teks = teks.replace(' '+i+' ',' ')
    return teks

In [16]:
i = 0
k1 = rmStop(DATA_SHORT[DATA_SHORT.index[i]])
k2 = rmStop(DATA_SHORTPos[DATA_SHORTPos.index[i]])
while (k1==k2):
    i+=1
    k1 = rmStop(DATA_SHORT[DATA_SHORT.index[i]])
    k2 = rmStop(DATA_SHORTPos[DATA_SHORTPos.index[i]])
print i

0


In [17]:
print DATA_SHORT[DATA_SHORT.index[i]]
print rmStops(DATA_SHORT[DATA_SHORT.index[i]])
print rmStops(DATA_SHORTPos[DATA_SHORTPos.index[i]])
DATA_SHORTPos.index[i]

saat masih lajang memang kebutuhan tak terlalu mendesak.
saat lajang kebutuhan mendesak.
lajang kebutuhan mendesak


648

Vektorisasi sudah termasuk menghilangkan karakter-karakter selain alphanumeric dan mengubah seluruh huruf menjadi huruf kecil. Kata 'ini', 'baik' , 'dengan' , dan 'sedang' hilang karena masuk dalam list stopwords

## Looking For Insight

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

def tableTop10(a,b,c):    
    def topKWords(docs,k):
        dummy  = docs.tolist()
        countv = CountVectorizer(stop_words=stopwords,decode_error='ignore')
        dummy  = countv.fit_transform(dummy)
        freqs = [(word, dummy.getcol(idx).sum()) for word, idx in countv.vocabulary_.items()]
        #sort from largest to smallest
        return [name for name,freq in sorted(freqs, key = lambda x: -x[1])[0:k]]
    
    top_word = pd.DataFrame([topKWords(a,10),
                             topKWords(b,10),
                             topKWords(c,10)]).transpose()
    return top_word

In [19]:
top_wordBerita1 = tableTop10(DATA_JUDUL,DATA_SHORT,DATA_LONG)
top_wordBerita1.columns = ['Judul','Short','Long']
top_wordBerita2 = tableTop10(DATA_JUDULPos,DATA_SHORTPos,DATA_LONGPos)
top_wordBerita2.columns = ['JudulPOSTag','ShortPOSTag','LongPOSTag']

In [20]:
top_wordBerita = [top_wordBerita1,top_wordBerita2]
top_wordBerita = pd.concat(top_wordBerita,axis=1)
print '10 Kata paling tinggi Frekuensinya'
top_wordBerita

10 Kata paling tinggi Frekuensinya


,Judul,Short,Long,JudulPOSTag,ShortPOSTag,LongPOSTag
0,air,pt,persen,rp,pt,rp
1,lion,indonesia,2016,air,rp,saham
2,com,tbk,saham,lion,indonesia,indonesia
3,jpnn,2016,indonesia,com,tbk,jakarta
4,miliar,persen,jakarta,jpnn,saham,pt
5,saham,saham,pt,miliar,laba,miliar
6,triliun,laba,miliar,saham,jakarta,perusahaan
7,laba,jakarta,triliun,laba,miliar,laba
8,mnc,miliar,perusahaan,mnc,bank,perseroan
9,rp,bank,laba,dividen,harga,air


Terlihat bahwa perbedaan hasilnya cukup baik. Hal ini terkait dengan hilangnya beberapa kata yang tidak mengandung makna yang signifikan terhadap informasi yang dikandungnya. Contoh beberapa kata tersebut adalah 'triliun' dan 'persen'. Kata 'triliun' jika didalam kalimat kutipan maka tidak dihilangkan'

In [21]:
onlyNVFromParagraph('"dengan repatriasi yang bisa mendatangkan dana rp 500 triliun, \
akan berpengaruh menggairahkan pasar modal, pasar uang dan perbankan. bahkan, bisa menggerakkan sektor riil (lapangan pekerjaan),"')

'repatriasi mendatangkan dana rp berpengaruh menggairahkan pasar modal pasar uang perbankan menggerakkan sektor riil lapangan pekerjaan'

In [22]:
pickle.dump( DATA_JUDULPos, open( "v1.6\DATA_JUDULpostag-v5a.p", "wb" ) )
pickle.dump( DATA_SHORTPos, open( "v1.6\DATA_SHORTpostag-v5a.p", "wb" ) )
pickle.dump( DATA_LONGPos, open( "v1.6\DATA_LONGpostag-v5a.p", "wb" ) )
pickle.dump( DATA_TARGET, open( "v1.6\DATA_TARGET-v5a.p", "wb" ) )

pickle.dump( DATA_JUDUL_skPos, open( "v1.6\DATA_JUDULpostag-v5c.p", "wb" ) )
pickle.dump( DATA_SHORT_skPos, open( "v1.6\DATA_SHORTpostag-v5c.p", "wb" ) )
pickle.dump( DATA_LONG_skPos, open( "v1.6\DATA_LONGpostag-v5c.p", "wb" ) )